# Lenet-5 Model with Deformable Convolution Layers

In [1]:
import torch
from torchvision import datasets
from torch.utils.data import DataLoader
from torchvision.transforms import ToTensor
import torch.nn as nn
import random
import numpy as np
from torchvision.ops import deform_conv2d
from torchvision.ops import DeformConv2d
device = torch.device('cuda')
from torchvision.transforms import v2
from torch.utils.data import default_collate

### Import Fashion MNIST Dataset

In [2]:
# Download and load the MNIST dataset
train_dataset = datasets.FashionMNIST(root='./data', train=True, download=True, transform= ToTensor())
test_dataset = datasets.FashionMNIST(root='./data', train=False, download=True, transform=ToTensor())

### Set Seed

In [3]:
seed = 0

random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
np.random.seed(seed)
torch.cuda.manual_seed_all(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

### Dataloaders with collate function for data augmentation using cutmix and mixup

In [4]:
batch_size = 50
cutmix = v2.CutMix(num_classes=10)
mixup = v2.MixUp(num_classes=10)
cutmix_or_mixup = v2.RandomChoice([cutmix, mixup])
def collate_fn(batch):
    return cutmix_or_mixup(*default_collate(batch))
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, collate_fn=collate_fn)
test_dataloader = DataLoader(test_dataset, batch_size= batch_size, shuffle=False)

### Define LeNet5 CNN with Deformable Convolution Layers

In [5]:

class LeNet5(nn.Module):
    def __init__(self, input_shape=(1, 28, 28), num_classes=10):
        super(LeNet5, self).__init__()
        torch.manual_seed(seed)
        self.offset1 = nn.Parameter(torch.randn(batch_size, 2*5*5, 24,24, requires_grad=True)*0.01)
        self.offset2 = nn.Parameter(torch.randn(batch_size, 2*5*5, 8,8, requires_grad=True)*0.01)
        
        self.deformconv1 = DeformConv2d(1, 32, kernel_size=5, stride=1)
        self.batchnorm1 = nn.BatchNorm2d(32)
        self.relu1 = nn.ReLU()
        self.maxpool1 = nn.MaxPool2d(kernel_size=2, stride=2)

        self.deformconv2 = DeformConv2d(32, 64, kernel_size=5, stride=1)
        self.batchnorm2 = nn.BatchNorm2d(64)
        self.relu2 = nn.ReLU()
        self.maxpool2 = nn.MaxPool2d(kernel_size=2, stride=2)
        
        self.classifier = nn.Sequential(
            nn.Linear(64 * 4 * 4, 120),
            nn.ReLU(),
            nn.Linear(120, 84),
            nn.ReLU(),
            nn.Linear(84, num_classes)
        )

    def forward(self, x):
        x = self.deformconv1(x, self.offset1)
        x = self.batchnorm1(x)
        x = self.relu1(x)
        x = self.maxpool1(x)
        
        x = self.deformconv2(x, self.offset2)
        x = self.batchnorm2(x)
        x = self.relu2(x)
        x = self.maxpool2(x)
        
        x = x.view(x.size(0), -1)
        x = self.classifier(x)
        return x

In [6]:
loss_fn = nn.CrossEntropyLoss()
device = torch.device('cuda')
model = LeNet5().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

### Train Function

In [7]:
def train(train_loader, model, epochs):
    for epoch in range(epochs):
        model.train()
        train_loss = 0.0
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            output = model(images)
            loss = loss_fn(output, labels)
            loss.backward()
            optimizer.step()
            train_loss += loss.item()
        # Calculate average training loss for the epoch
        avg_train_loss = train_loss / len(train_dataloader)
        print(f'Epoch {epoch + 1}/{epochs}, Train Loss: {avg_train_loss:.4f}')

### Test Function

In [8]:
def test(test_loader, model):
    model.eval()
    correct = 0
    total = 0
    test_loss = 0

    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            output = model(images)
            loss = loss_fn(output, labels)
            test_loss += loss.item()

            _, predicted = torch.max(output, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    # Calculate average validation loss and accuracy for the epoch
    avg_test_loss = test_loss / len(test_dataloader)
    accuracy = 100 * (correct / total)

    return avg_test_loss, accuracy

### Commence Training

In [9]:
train(train_dataloader, model, 50)

Epoch 1/50, Train Loss: 1.1531
Epoch 2/50, Train Loss: 1.0092
Epoch 3/50, Train Loss: 0.9736
Epoch 4/50, Train Loss: 0.9469
Epoch 5/50, Train Loss: 0.9127
Epoch 6/50, Train Loss: 0.9086
Epoch 7/50, Train Loss: 0.9100
Epoch 8/50, Train Loss: 0.8880
Epoch 9/50, Train Loss: 0.8739
Epoch 10/50, Train Loss: 0.8914
Epoch 11/50, Train Loss: 0.8762
Epoch 12/50, Train Loss: 0.8514
Epoch 13/50, Train Loss: 0.8646
Epoch 14/50, Train Loss: 0.8575
Epoch 15/50, Train Loss: 0.8581
Epoch 16/50, Train Loss: 0.8448
Epoch 17/50, Train Loss: 0.8432
Epoch 18/50, Train Loss: 0.8400
Epoch 19/50, Train Loss: 0.8362
Epoch 20/50, Train Loss: 0.8331
Epoch 21/50, Train Loss: 0.8325
Epoch 22/50, Train Loss: 0.8318
Epoch 23/50, Train Loss: 0.8362
Epoch 24/50, Train Loss: 0.8268
Epoch 25/50, Train Loss: 0.8148
Epoch 26/50, Train Loss: 0.8280
Epoch 27/50, Train Loss: 0.8297
Epoch 28/50, Train Loss: 0.8232
Epoch 29/50, Train Loss: 0.8237
Epoch 30/50, Train Loss: 0.8268
Epoch 31/50, Train Loss: 0.8226
Epoch 32/50, Trai

### Test trained model on unseen data

In [10]:
avg_test_loss, acc = test(test_dataloader, model)

In [11]:
print("Accuracy on modified model: " + str(acc) + "%")
print("Average test loss on modified model: " + str(avg_test_loss) + "%")

Accuracy on modified model: 92.33%
Average test loss on modified model: 0.2803454554453492%
